In [3]:
from resolve_api_samples.Examples.python_get_resolve import GetResolve
from pprint import pprint
import sys

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()

sys.path.append("/Users/thom/PycharmProjects/quick-import/resolve_api_samples")
sys.path

['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug',
 '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev',
 '/Users/thom/PycharmProjects/quick-import/src',
 '/Users/thom/PycharmProjects/quick-import',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/thom/PycharmProjects/quick-import/venv/lib/python3.10/site-packages',
 '/Users/thom/PycharmProjects/quick-import/resolve_api_samples',
 '/Users/thom/PycharmProjects/quick-import/resolve_api_samples']